# TinyML Handwritten Digit Recognizer

## Dataset: MNIST (handwritten single digit 28x28)

In [1]:
!pip install numpy tensorflow

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, utils
from tensorflow.keras import models, layers, losses

## Prepare Dataset

In [3]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

In [4]:
# data normalized
x_train = x_train/255.
x_test  = x_test/255.

# reshape for model input
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [5]:
# encoding y data
y_train = utils.to_categorical(y_train)
y_test  = utils.to_categorical(y_test)

print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


## Build Model

In [6]:
input_shape = (28,28,1) # img_rows, img_colums, color_channels
num_classes = 10

In [7]:
## Build Model
inputs = layers.Input(shape=input_shape)
x = layers.Conv2D(16, kernel_size = (3, 3), activation = 'relu', padding = 'same')(inputs)
x = layers.MaxPool2D(pool_size = (2, 2))(x)
# 2nd Conv layer        
x = layers.Conv2D(16, kernel_size = (3, 3), activation = 'relu', padding = 'same')(x)
x = layers.MaxPool2D(pool_size = (2, 2))(x)
# Fully Connected layer        
x = layers.Flatten()(x)
x = layers.Dense(64)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 16)        2320      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 784)               0     

2022-07-18 15:03:07.795129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train Model

In [9]:
# Train Model
history = model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))

Epoch 1/20
469/469 [==============================] - 12s 24ms/step - loss: 0.2871 - accuracy: 0.9163 - val_loss: 0.0828 - val_accuracy: 0.9750
Epoch 2/20
469/469 [==============================] - 12s 25ms/step - loss: 0.0813 - accuracy: 0.9750 - val_loss: 0.0637 - val_accuracy: 0.9796
Epoch 3/20
469/469 [==============================] - 13s 27ms/step - loss: 0.0617 - accuracy: 0.9811 - val_loss: 0.0515 - val_accuracy: 0.9842
Epoch 4/20
469/469 [==============================] - 14s 29ms/step - loss: 0.0506 - accuracy: 0.9847 - val_loss: 0.0443 - val_accuracy: 0.9864
Epoch 5/20
469/469 [==============================] - 15s 32ms/step - loss: 0.0437 - accuracy: 0.9858 - val_loss: 0.0422 - val_accuracy: 0.9869
Epoch 6/20
469/469 [==============================] - 15s 32ms/step - loss: 0.0390 - accuracy: 0.9875 - val_loss: 0.0443 - val_accuracy: 0.9854
Epoch 7/20
469/469 [==============================] - 15s 32ms/step - loss: 0.0362 - accuracy: 0.9884 - val_loss: 0.0416 - val_accuracy:

## Evaluate Model

In [10]:
# Evaluate Model
score = model.evaluate(x_test, y_test)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

313/313 [==============================] - 4s 12ms/step - loss: 0.0473 - accuracy: 0.9878
Test loss:  0.04726804047822952
Test accuracy:  0.9878000020980835


## Test Model

In [11]:
y_pred = model.predict(x_test[0].reshape(-1,28,28,1)).argmax(axis=1)
print(y_pred)

1/1 [==============================] - 0s 75ms/step
[7]


## TinyML

In [12]:
!pip install tinymlgen

### export model

In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("mnist_cnn_quantized.tflite", "wb").write(tflite_model)

!xxd -i mnist_cnn_quantized.tflite > mnist_cnn_quantized.cc

INFO:tensorflow:Assets written to: /var/folders/w1/9lc2_mk113z9m56r_3nvjmrw0000gn/T/tmpqe4yy4ou/assets


INFO:tensorflow:Assets written to: /var/folders/w1/9lc2_mk113z9m56r_3nvjmrw0000gn/T/tmpqe4yy4ou/assets
2022-07-18 15:08:07.841114: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-07-18 15:08:07.841128: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-07-18 15:08:07.841858: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /var/folders/w1/9lc2_mk113z9m56r_3nvjmrw0000gn/T/tmpqe4yy4ou
2022-07-18 15:08:07.843895: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-07-18 15:08:07.843907: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /var/folders/w1/9lc2_mk113z9m56r_3nvjmrw0000gn/T/tmpqe4yy4ou
2022-07-18 15:08:07.849473: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-07-18 15:08:07.852722: I tensorflow/cc/saved_model/load

In [14]:
from tinymlgen import port
c_code = port(model, variable_name='mnist_cnn', pretty_print=True, optimize=False) 

with open('mnist_cnn.h', 'w') as f:
    print(c_code, file=f)

INFO:tensorflow:Assets written to: /var/folders/w1/9lc2_mk113z9m56r_3nvjmrw0000gn/T/tmpxilly6cp/assets


INFO:tensorflow:Assets written to: /var/folders/w1/9lc2_mk113z9m56r_3nvjmrw0000gn/T/tmpxilly6cp/assets
2022-07-18 15:08:10.408302: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-07-18 15:08:10.408320: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-07-18 15:08:10.408452: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /var/folders/w1/9lc2_mk113z9m56r_3nvjmrw0000gn/T/tmpxilly6cp
2022-07-18 15:08:10.415953: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-07-18 15:08:10.415973: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /var/folders/w1/9lc2_mk113z9m56r_3nvjmrw0000gn/T/tmpxilly6cp
2022-07-18 15:08:10.422676: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-07-18 15:08:10.476793: I tensorflow/cc/saved_model/loader.cc:212] Running initialization

In [15]:
!sed -i '' -E 's/ +$//g' mnist_cnn.h

### export a test data (uint8)

In [16]:
# load raw x_test
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
    
with open('x_test.h', 'w') as f:
    print("unsigned char x_test_dat[784] = {", file=f)    
    for i in range(28):
        s = "       "
        for j in range(28):
            s+=str(x_test[0][i][j])+', ' # select x_test[0]
        print(s, file=f)
    print("};", file=f)
f.close()

In [17]:
!sed -i '' -E 's/ +$//g' x_test.h

In [18]:
!cat x_test.h

unsigned char x_test_dat[784] = {
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 84, 185, 159, 151, 60, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 222, 254, 254, 254, 254, 241, 198, 198, 198, 198, 198, 198, 198, 198, 170, 52, 0, 0, 0, 0, 0, 0,

       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 66, 14, 67, 67, 67, 59, 21, 236, 254, 106, 0, 0, 0, 0, 0, 0,
 